In [1]:
from metaworld.envs.mujoco.env_dict import ALL_V1_ENVIRONMENTS, ALL_V2_ENVIRONMENTS
from metaworld.policies import *
from tests.metaworld.envs.mujoco.sawyer_xyz.utils import check_success

In [ ]:
config = {
    'envs': [
        'button-press-topdown-v1'
    ],

}
